# Import requirements

In [ ]:
import numpy
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os
import pickle
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Step 1: Open csv

In [ ]:
path_to_csv = "../dataset/Tweets.csv"
dataframe = pd.read_csv(path_to_csv)
dataframe = dataframe[dataframe['airline_sentiment_confidence'] >= 0.75].drop_duplicates()
filtered_df = dataframe.filter(items=["text","airline_sentiment"])
filtered_df

# Step 2: Extract phrases and labels

In [ ]:
training_phrases = filtered_df["text"].to_numpy()
training_labels = filtered_df["airline_sentiment"].to_numpy()

# Step 3: Plot label distribution

In [ ]:
def plot_label_dist(labels):
    label_dist, label_dist_count = numpy.unique(labels, return_counts = True)
    label_dist = label_dist.astype("str")

    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.set_ylabel('Count')
    ax.set_title('Label distribution')
    ax.bar(label_dist,label_dist_count)
    plt.show()

plot_label_dist(training_labels)

# Step 4: Text preprocessor
- lowercase
- remove punctuations
- remove white spaces

In [ ]:
def text_preprocessor(training_phrases):
    """
    input: array
    Takes in an array with sentences & cleans the sentences
    output: array """
    clean_tps = []
    for sentence in training_phrases:
        # Lowercase
        sentence = sentence.lower()
        # Remove white spaces
        sentence = re.sub(' +', ' ',sentence).strip()
        # Append cleaned sentences
        clean_tps.append(sentence)

    return clean_tps

clean_tps = text_preprocessor(training_phrases)

print(f"Before preprocessor:\n{training_phrases[20]}\n\nAfter preprocessor: \n{clean_tps[20]}")

# ======================= Featurization =======================

# Step 5: Countvectorizer

In [ ]:
vectorizer = CountVectorizer(max_features=5000).fit(clean_tps)
sparse_features = vectorizer.transform(clean_tps)

# Checkpoint 

In [ ]:
print("Shape of features: {}".format(sparse_features.shape))
print("Vectorized example: {}".format(sparse_features[0].toarray()))

# Step 6: Split data in train and test

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(sparse_features, 
                                                  training_labels,
                                                  test_size = 0.30, 
                                                  random_state = 1, 
                                                  stratify = training_labels)

# =============== Classification algorithm ===============

# Step 7: SVM model

In [ ]:
clf = SVC(kernel='linear', 
          probability=True)

clf.fit(x_train,y_train)
y_pred = clf.predict(x_val)
print(accuracy_score(y_val,y_pred))

# Step 8: Model evaluation

In [ ]:
print(classification_report(y_val,y_pred))
print(confusion_matrix(y_val,y_pred))
print()
print(accuracy_score(y_val,y_pred))

# =============== Save objects to disk ===============
- count vectorizer
- trained model

## Paths

In [ ]:
path_to_model = "../models/"
path_to_features = "../featurizers/"
feature_name = 'sparse_features'
model_name = 'svm'

## Save as pickle

In [ ]:
def save_as_pickle(obj, dir_name, file_name):
    is_exist = os.path.exists(dir_name)
    if not is_exist:
        os.makedirs(dir_name)
    pickle.dump(obj, open(dir_name+file_name, 'wb'))
    
save_as_pickle(vectorizer, path_to_features, feature_name)
print("feature object is succesfully saved...")
save_as_pickle(clf, path_to_model, model_name)
print("model object is successfully saved...")